<a href="https://colab.research.google.com/github/SoniaRFdz/TFG/blob/main/TFG6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random

# Fijar semillas aleatorias para reproducibilidad (Sin esto nunca se consiguen iguales valores)
seed = 42
tf.random.set_seed(seed)
random.seed(seed)

# Cargar los datos
data = pd.read_csv('datos_flujo_laminar_simulado.csv')

# Definir características (X) y objetivo (y)
X = data[['Diametro (m)', 'Presion (Pa)', 'Velocidad (m/s)']].values
y = data['Flujo Volumetrico (m³/s)'].values

# Escalar los datos de salida entre 0 y 1 (Hay que modificar los datos de flujo porque si no el modelo da problemas de convergencia)
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y.reshape(-1, 1))

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características de entrada (Estandarización de los datos)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Definir el modelo de red neuronal
model = Sequential([
    Input(shape=(3,)),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    Dropout(0.1),
    Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    Dense(1)
])

# Compilar el modelo
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Configurar EarlyStopping (Para evitar sobreajustes)
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Entrenar el modelo (verbose evita que muestre los valores para todas las etapas)
history = model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_split=0.2,verbose=0, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test)
print(f"Error cuadrático medio en el conjunto de prueba: {loss}")

# Hacer predicciones
predicciones = model.predict(X_test)

# Invertir la escala de las predicciones para comparar con los valores originales (modifica los datos de salida como al inicio)
predicciones_originales = scaler_y.inverse_transform(predicciones)
y_test_originales = scaler_y.inverse_transform(y_test)

# Mostrar algunas predicciones y valores reales para comparación
for i in range(5):
    print(f"Predicción: {predicciones_originales[i][0]}, Valor real: {y_test_originales[i][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0038
Error cuadrático medio en el conjunto de prueba: 0.00377859640866518
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicción: 5.975983619689941, Valor real: 5.303646535267333
Predicción: -0.5713512301445007, Valor real: 0.0014726215563702
Predicción: -0.06908640265464783, Valor real: 0.0007363107781851
Predicción: 5.230179786682129, Valor real: 5.026548245743668
Predicción: 1.6073483228683472, Valor real: 1.5904312808798335
